## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf
tf.enable_eager_execution

<function tensorflow.python.framework.ops.enable_eager_execution>

In [0]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/R6/L1/prices.csv')

### Check all columns in the dataset

In [6]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [7]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [0]:
data.drop('date',axis=1, inplace=True)
data.drop('symbol',axis=1, inplace=True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [10]:
data.shape

(851264, 5)

### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
newData=data[0:1000]

In [12]:
newData.shape

(1000, 5)

In [13]:
newData.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [14]:
newData['volume']=newData['volume']/1000000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
newData.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X = newData.drop('volume', axis=1)

Y = newData[['volume']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.20, random_state=7)

#### Convert Training and Test Data to numpy float32 arrays


In [17]:
print(X_train.dtypes)
print(y_train.dtypes)
print(X_test.dtypes)
print(y_test.dtypes)

open     float64
close    float64
low      float64
high     float64
dtype: object
volume    float64
dtype: object
open     float64
close    float64
low      float64
high     float64
dtype: object
volume    float64
dtype: object


In [0]:
import numpy as np
train_x = np.array(X_train).astype('float32')
train_y = np.array(y_train).astype('float32')
test_x = np.array(X_test).astype('float32')
test_y = np.array(y_test).astype('float32')

In [19]:
print(train_x.dtype)
print(train_y.dtype)
print(test_x.dtype)
print(test_y.dtype)

float32
float32
float32
float32


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x =  transformer.fit_transform(test_x)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

In [22]:
b.shape

TensorShape([Dimension(1)])

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [26]:
print(train_x.shape)
print(train_y.shape)
print(train_x.dtype)
print(np.array(train_y).dtype)

(800, 4)
(800, 1)
float32
float32


In [27]:
for i in range(100):
    
    w, b = train(train_x, train_y, w, b)
    print('Current Loss on iteration', i, loss(train_y, prediction(train_x, w, b)))

Current Loss on iteration 0 Tensor("Mean_1:0", shape=(), dtype=float32)
Current Loss on iteration 1 Tensor("Mean_3:0", shape=(), dtype=float32)
Current Loss on iteration 2 Tensor("Mean_5:0", shape=(), dtype=float32)
Current Loss on iteration 3 Tensor("Mean_7:0", shape=(), dtype=float32)
Current Loss on iteration 4 Tensor("Mean_9:0", shape=(), dtype=float32)
Current Loss on iteration 5 Tensor("Mean_11:0", shape=(), dtype=float32)
Current Loss on iteration 6 Tensor("Mean_13:0", shape=(), dtype=float32)
Current Loss on iteration 7 Tensor("Mean_15:0", shape=(), dtype=float32)
Current Loss on iteration 8 Tensor("Mean_17:0", shape=(), dtype=float32)
Current Loss on iteration 9 Tensor("Mean_19:0", shape=(), dtype=float32)
Current Loss on iteration 10 Tensor("Mean_21:0", shape=(), dtype=float32)
Current Loss on iteration 11 Tensor("Mean_23:0", shape=(), dtype=float32)
Current Loss on iteration 12 Tensor("Mean_25:0", shape=(), dtype=float32)
Current Loss on iteration 13 Tensor("Mean_27:0", shap

### Get the shapes and values of W and b

In [28]:
print('Weights:\n', w)
print('Bias:\n',b)

Weights:
 Tensor("sub_397:0", shape=(4, 1), dtype=float32)
Bias:
 Tensor("sub_398:0", shape=(1,), dtype=float32)


In [29]:
train_x[0]

array([0.49873042, 0.5009242 , 0.49385527, 0.5064088 ], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

In [0]:
current_prediction2 = prediction(test_x, w, b)

In [31]:
current_prediction2.shape

TensorShape([Dimension(200), Dimension(1)])

In [0]:
with tf.Session() as sess:  
  opVal=current_prediction2.eval()

In [33]:
opVal[0:1]

array([[5.3023324]], dtype=float32)

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
irisDF=pd.read_csv('/content/drive/My Drive/Colab Notebooks/R6/L1/Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [35]:
irisDF.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [36]:
import pandas as pd
df_occ = pd.get_dummies(data = irisDF['Species'], dummy_na = True)
df_occ = df_occ.rename(columns = {'Iris-setosa' : 'Iris_setosa','Iris-versicolor' : 'Iris_versicolor','Iris-virginica' : 'Iris_virginica'})
dev = pd.concat([irisDF, df_occ.iloc[:, 0:3]], axis = 1)
dev.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Iris_setosa,Iris_versicolor,Iris_virginica
0,1,5.1,3.5,1.4,0.2,Iris-setosa,1,0,0
1,2,4.9,3.0,1.4,0.2,Iris-setosa,1,0,0
2,3,4.7,3.2,1.3,0.2,Iris-setosa,1,0,0
3,4,4.6,3.1,1.5,0.2,Iris-setosa,1,0,0
4,5,5.0,3.6,1.4,0.2,Iris-setosa,1,0,0


In [0]:
dev.drop('Species',axis=1, inplace=True)

dev.drop('Id',axis=1, inplace=True)

### Splitting the data into feature set and target set

In [38]:
dev.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Iris_setosa,Iris_versicolor,Iris_virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


In [0]:
X = dev.drop(['Iris_setosa','Iris_versicolor','Iris_virginica'], axis=1)

Y = dev[['Iris_setosa','Iris_versicolor','Iris_virginica']]

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [40]:
X.shape

(150, 4)

In [0]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=.20, random_state=7)

In [42]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 120x4 to 480
model.add(tf.keras.layers.Dense(1,input_shape=(4,)))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Comile the model
# Another option is binary_crossentropy
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### Model Training 

In [43]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(120, 4)
(120, 3)
(30, 4)
(30, 3)


In [44]:
model.fit(trainX, trainY, 
          validation_data=(testX, testY), 
          epochs=100,
          batch_size=trainX.shape[0])

Train on 120 samples, validate on 30 samples
Epoch 1/100
120/120 [==============================] - 0s 4ms/sample - loss: 1.7575 - acc: 0.2833 - val_loss: 5.6426 - val_acc: 0.4000
Epoch 2/100
120/120 [==============================] - 0s 77us/sample - loss: 1.7235 - acc: 0.2917 - val_loss: 5.6166 - val_acc: 0.4000
Epoch 3/100
120/120 [==============================] - 0s 74us/sample - loss: 1.6906 - acc: 0.3083 - val_loss: 5.5889 - val_acc: 0.4000
Epoch 4/100
120/120 [==============================] - 0s 69us/sample - loss: 1.6590 - acc: 0.3333 - val_loss: 5.5588 - val_acc: 0.4000
Epoch 5/100
120/120 [==============================] - 0s 69us/sample - loss: 1.6291 - acc: 0.3417 - val_loss: 5.5260 - val_acc: 0.4000
Epoch 6/100
120/120 [==============================] - 0s 70us/sample - loss: 1.6011 - acc: 0.3417 - val_loss: 5.4902 - val_acc: 0.4000
Epoch 7/100
120/120 [==============================] - 0s 73us/sample - loss: 1.5750 - acc: 0.3500 - val_loss: 5.4516 - val_acc: 0.4000
Epoc

### Model Prediction

In [0]:
pred = model.predict(testX)

In [46]:
pred

array([[0.01996436, 0.9007497 , 0.07928604],
       [0.04242911, 0.83211917, 0.1254517 ],
       [0.11657514, 0.6612777 , 0.22214723],
       [0.03473837, 0.8539939 , 0.11126772],
       [0.06474536, 0.7745599 , 0.16069463],
       [0.22157931, 0.4784816 , 0.29993913],
       [0.00905475, 0.94259506, 0.04835007],
       [0.04502068, 0.82502514, 0.12995417],
       [0.15723522, 0.5847447 , 0.25802013],
       [0.01420367, 0.92164207, 0.06415433],
       [0.00937678, 0.94119537, 0.04942783],
       [0.01823807, 0.90679705, 0.07496484],
       [0.10341906, 0.68805665, 0.20852426],
       [0.00206168, 0.9790949 , 0.01884338],
       [0.11156473, 0.6713444 , 0.2170909 ],
       [0.01800541, 0.9076247 , 0.0743698 ],
       [0.0109872 , 0.9343974 , 0.0546154 ],
       [0.01202029, 0.9301928 , 0.05778681],
       [0.1122916 , 0.6698742 , 0.21783417],
       [0.16714446, 0.56730866, 0.26554692],
       [0.03942926, 0.84049684, 0.12007395],
       [0.00390891, 0.9677242 , 0.02836687],
       [0.

In [47]:
pred[0].argmax()

1

In [48]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5         
_________________________________________________________________
batch_normalization (BatchNo (None, 1)                 4         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 6         
Total params: 15
Trainable params: 13
Non-trainable params: 2
_________________________________________________________________


### Save the Model

In [0]:
model.save('R6_Int_PSM.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [0]:
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(50, activation='sigmoid'))

In [0]:
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(3, activation='sigmoid'))

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [53]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5         
_________________________________________________________________
batch_normalization (BatchNo (None, 1)                 4         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 6         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                200       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 153       
Total params: 368
Trainable params: 366
Non-trainable params: 2
_________________________________________________________________


In [55]:
model.fit(trainX,trainY,          
          validation_data=(testX,testY),
          epochs=200,
          batch_size=100)

Train on 120 samples, validate on 30 samples
Epoch 1/200
120/120 [==============================] - 0s 179us/sample - loss: 1.0893 - acc: 0.3583 - val_loss: 1.1052 - val_acc: 0.2333
Epoch 2/200
120/120 [==============================] - 0s 137us/sample - loss: 1.0893 - acc: 0.3583 - val_loss: 1.1047 - val_acc: 0.2333
Epoch 3/200
120/120 [==============================] - 0s 146us/sample - loss: 1.0893 - acc: 0.3583 - val_loss: 1.1050 - val_acc: 0.2333
Epoch 4/200
120/120 [==============================] - 0s 128us/sample - loss: 1.0893 - acc: 0.3583 - val_loss: 1.1046 - val_acc: 0.2333
Epoch 5/200
120/120 [==============================] - 0s 121us/sample - loss: 1.0892 - acc: 0.3583 - val_loss: 1.1048 - val_acc: 0.2333
Epoch 6/200
120/120 [==============================] - 0s 124us/sample - loss: 1.0892 - acc: 0.3583 - val_loss: 1.1047 - val_acc: 0.2333
Epoch 7/200
120/120 [==============================] - 0s 132us/sample - loss: 1.0892 - acc: 0.3583 - val_loss: 1.1045 - val_acc: 0.2

In [0]:
# After adding two hidden layers, the accuracy is going down. Please let me know what further can be done to improve it